In [1]:
# apps = ["platformer_2k"]
apps = [
    'pbrtexture_2k', 'instancing_2k', 'render_passes_2k', 
    'sponza_2k', 'materials_2k', 'platformer_2k',
    'demo_2k', 
    # 'pbrtexture_2k_lod0', 'instancing_2k_lod0', 'render_passes_2k_lod0',
    'pbrtexture_4k', 'instancing_4k', 'render_passes_4k', 
    'sponza_4k', 'materials_4k', 'platformer_4k',
    'demo_4k',
]
suite = 'vulkan'
# suite = 'vulkan-correlation'

addons = ''
# addons += '-perf_l1'
# addons += '-skip_l2'
# addons += '-mps_sm8'
addons += '-VISUAL'


configs = ''
configs += "ORIN-SASS" + addons + ','
configs += "ORIN-SASS-concurrent-fg" + addons + ','
configs += "RTX3070-SASS" + addons + ','
configs += "RTX3070-SASS-concurrent-fg" + addons + ','

configs += "ORIN-SASS-concurrent-mps_sm8" + addons + ','
configs += "ORIN-SASS-concurrent-MIG-mps_sm8" + addons + ','
configs += "ORIN-SASS-concurrent-fg-dynamic_sm3" + addons + ','
configs += "RTX3070-SASS-concurrent-mps_sm8" + addons + ','
configs += "RTX3070-SASS-concurrent-MIG-mps_sm8" + addons + ','
configs += "RTX3070-SASS-concurrent-fg-dynamic_sm3" + addons + ','

configs += "ORIN-SASS-concurrent-mps_sm8-utility" + addons + ','
configs += "ORIN-SASS-concurrent-fg-dynamic_sm3-utility" + addons + ','
configs += "RTX3070-SASS-concurrent-mps_sm8-utility" + addons + ','
configs += "RTX3070-SASS-concurrent-fg-dynamic_sm3-utility" + addons + ','

configs += "ORIN-SASS-concurrent-fg-dynamic_sm3-slicer" + addons + ','
configs += "RTX3070-SASS-concurrent-fg-dynamic_sm3-slicer" + addons + ','

# configs += "ORIN-SASS-invalidate_l2" + addons + ','
# configs += "ORIN-SASS-concurrent-fg" + addons + ','
# configs += "ORIN-SASS-concurrent-fg-invalidate_l2" + addons + ','
# configs += "ORIN-SASS-concurrent-fg-best" + addons + ','
# configs += "ORIN-SASS-concurrent-fg-invalidate_l2-best" + addons + ','
# configs += "ORIN-SASS-concurrent-fg-utility-best" + addons + ','
# configs += "ORIN-SASS-concurrent-fg-slicer" + addons + ','
# configs += "ORIN-SASS-concurrent-fg-slicer-best" + addons + ','
# configs += "ORIN-SASS-concurrent-fg-slicer-invalidate_l2" + addons + ','
# configs += "ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best" + addons + ','

# for i in range(12,16):
    # configs += "ORIN-SASS-concurrent-" + str(i) +'GR' + addons + ','
# for i in range(12,16):
    # configs += "ORIN-SASS-concurrent-" + str(i) +'GR-MIG' + addons + ','
# for i in range(1,14):
#     configs += "ORIN-SASS-concurrent-" + str(i) +'GR-finegrain' +  addons + ','


configs = configs[:-1]
print(configs)



ORIN-SASS-VISUAL,ORIN-SASS-concurrent-fg-VISUAL,RTX3070-SASS-VISUAL,RTX3070-SASS-concurrent-fg-VISUAL,ORIN-SASS-concurrent-mps_sm8-VISUAL,ORIN-SASS-concurrent-MIG-mps_sm8-VISUAL,ORIN-SASS-concurrent-fg-dynamic_sm3-VISUAL,RTX3070-SASS-concurrent-mps_sm8-VISUAL,RTX3070-SASS-concurrent-MIG-mps_sm8-VISUAL,RTX3070-SASS-concurrent-fg-dynamic_sm3-VISUAL,ORIN-SASS-concurrent-mps_sm8-utility-VISUAL,ORIN-SASS-concurrent-fg-dynamic_sm3-utility-VISUAL,RTX3070-SASS-concurrent-mps_sm8-utility-VISUAL,RTX3070-SASS-concurrent-fg-dynamic_sm3-utility-VISUAL,ORIN-SASS-concurrent-fg-dynamic_sm3-slicer-VISUAL,RTX3070-SASS-concurrent-fg-dynamic_sm3-slicer-VISUAL


In [2]:
import os
from datetime import datetime
# save current working directory
os.chdir("/home/tgrogers-raid/a/pan251/accel-sim-framework/util/graphics")
cwd = os.getcwd()
# change cwd to accel-sim-framework
root = os.path.join(cwd,"../../")
os.chdir(root)

today = datetime.now()
d = today.strftime("%Y%m%d-%H%M")
# open run.sh
run_sh = open(os.path.join(root,"run.sh"), 'w')
run_sh.write("#!/bin/bash\n")
run_sh.write("source ./gpu-simulator/setup_environment.sh\n")

for app in apps:
    # cmd = "./util/job_launching/run_simulations.py -B vulkan:" + app + " -C " + configs + " -T ./hw_run/traces/vulkan/ -N run-" + d + "-" + app + "\n"
    cmd = "./util/job_launching/run_simulations.py -B " + suite + ":" + app + " -C " + configs + " -T ./hw_run/traces/vulkan/ -N run-" + d + "-" + app + "\n"
    run_sh.write(cmd)
run_sh.close()

run_sh = open(os.path.join(root,"collect.sh"), 'w')
run_sh.write("#!/bin/bash\n")
run_sh.write("name=\"-N run-" + d + "\"\n")
run_sh.write('trap "trap - SIGTERM && kill -- -$$" SIGINT SIGTERM EXIT\n')
for app in apps:
    cmd = "./util/job_launching/get_stats.py -k -R $name-" + app + " > " + app + ".csv &\n"
    run_sh.write(cmd)
# run_sh.write("grep -r -c --include='*.o*' \"relaunching graphics kernels\" ./sim_run_11.0 > relaunch_graphics.txt &\n")
# run_sh.write("grep -r -c --include='*.o*' \"relaunching compute kernels\" ./sim_run_11.0 > relaunch_compute.txt &\n")
run_sh.write("wait < <(jobs -p)\n")
run_sh.close()
os.chdir(cwd)

print("run-"+d)

run-20240601-2201


In [17]:
import os

name = "run-20231030-1705"
cmd = "./util/job_launching/get_stats.py -k -R -N " + name + " | tee per.kernel.stats.csv"

print(cmd)
# os.system(cmd)


./util/job_launching/get_stats.py -k -R -N run-20231030-1705 | tee per.kernel.stats.csv


In [7]:
import pandas as pd
import numpy as np
import os
# open per.kernel.stats.csv as dataframe
dir = os.getenv("HOME") + "/accel-sim-framework/"
# csv = "per.kernel.stats.csv"
csv = "per.kernel.stats.csv"
g_count = 48

### Loop the data lines
with open(dir + csv, 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

temp_f.close()
### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]
    
file = pd.read_csv(dir + csv, header=None, delimiter=",", names=column_names)
file = file.fillna(0)
stat_to_row = {}
for rowi in range(0,file.shape[0]):
    # get the row
    row = file.iloc[rowi]
    if ('---------' in row[0]):
        print(file.iloc[rowi+1][0])
        stat_to_row[file.iloc[rowi+1][0]] = rowi+1

Accel-Sim-build
GPGPU-Sim-build
gpu_tot_sim_insn\s*=\s*(.*)
gpu_tot_sim_cycle\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[MISS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[TEXTURE_ACC_R\]\[MISS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[TEXTURE_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MSHR_HIT\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*)
\s+Total_core_cach

In [12]:
# get cycle rows
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[TEXTURE_ACC_R\]\[MISS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[TEXTURE_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[MISS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MSHR_HIT\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
rowi = stat_to_row['gpu_sim_cycle\s*=\s*(.*)']

diff = stat_to_row['GPGPU-Sim-build'] - stat_to_row["Accel-Sim-build"]
# print(diff)
# kernel_names = ["NO_ARGS--MESA", "corners--MESA", "fisheye--MESA", "refined--MESA", "klt_tracker--MESA", "all--MESA"]
kernel_names = ["all--MESA"]

configs_row = file.iloc[rowi+1]
# new dataframe
all_g = pd.DataFrame()
ga = pd.DataFrame()
ca = pd.DataFrame()
ga["CFG"] = file.iloc[:,0]
ca["CFG"] = file.iloc[:,0]
all_c = pd.DataFrame()
for app in apps:
    for kname in kernel_names:
        for index, column in configs_row.iteritems():
            if app in column:
                if kname in column:
                    ga[column] = file.iloc[:,index]
                elif  "MESA" not in column:
                    ca[column] = file.iloc[:,index]
        name = app + "/" + kname.replace('--MESA','')

for app in apps:
    found = False
    start = 0
    end = 0
    for idx, x in enumerate(list(ga.columns.values)):
        if app in x:
            if found == False:
                start = idx
                found = True
            end = idx+1
    all_g[app] = ga.iloc[rowi+2:rowi+diff-1,start:end].astype(int).sum(axis=1)

for app in apps:
    found = False
    start = 0
    end = 0
    for idx, x in enumerate(list(ca.columns.values)):
        if app in x and "all" in x:
            if found == False:
                start = idx
                found = True
            end = idx+1
    all_c[app] = ca.iloc[rowi+2:rowi+diff-1,start:end].astype(int).sum(axis=1)

all_g


,sponza_2k,sponza_4k
153,8004134,18076075
154,8355692,18356808
155,8281516,18260616


In [11]:
# get cycle rows
rowi = stat_to_row['gpu_sim_cycle\s*=\s*(.*)']

diff = stat_to_row['GPGPU-Sim-build'] - stat_to_row["Accel-Sim-build"]
# print(diff)
# kernel_names = ["NO_ARGS--MESA", "corners--MESA", "fisheye--MESA", "refined--MESA", "klt_tracker--MESA", "all--MESA"]
kernel_names = [ "all--MESA"]

configs_row = file.iloc[rowi+1]
# new dataframe
all_g = pd.DataFrame()
ga = pd.DataFrame()
ca = pd.DataFrame()
ga["CFG"] = file.iloc[:,0]
ca["CFG"] = file.iloc[:,0]
all_c = pd.DataFrame()
for app in apps:
    for kname in kernel_names:
        for index, column in configs_row.iteritems():
            if app in column:
                if kname in column:
                    ga[column] = file.iloc[:,index]
                elif  "MESA" not in column:
                    ca[column] = file.iloc[:,index]
        name = app + "/" + kname.replace('--MESA','')
# all_c
ca = ca.iloc[1:]
ga = ga.iloc[1:]
ca.set_index("CFG", inplace=True)
ga.set_index("CFG", inplace=True)
ga.to_csv("output_ga.csv")
ca.to_csv("output_ca.csv")

In [12]:
import pandas as pd
import numpy as np
import os
# open per.kernel.stats.csv as dataframe
dir = os.getenv("HOME") + "/accel-sim-framework/"
# csv = "per.kernel.stats.csv"
csv = "utility.csv"
    
file = pd.read_csv(dir + csv, header=None, delimiter=",")
file.columns = file.iloc[0]
file = file[1:].astype(float)
# reverse the row order
# file = file.iloc[::-1]

# get num of rows
ratio = 10766215/3251448
# if ratio < 10.0:
ratio = 1
num_rows = file.shape[0]
for i in range(num_rows):
    gr_rows = i
    cp_rows = 16 - i
    utility = 0
    for index,row in file.iterrows():
        real_index = index
        gr = row[1]/ratio
        cp = row[2]
        if real_index < gr_rows:
            utility += gr
            # print("gr", row[1])
        if real_index < cp_rows:
            utility += cp
            # print("cp", row[2])
    print(utility,", ")
    # print("gr =", i, utility)
file


0.0 , 
0.0 , 
2013442.0 , 
2238648.0 , 
2310280.0 , 
2354550.0 , 
2378180.0 , 
2387662.0 , 
2391082.0 , 
2393626.0 , 
2395330.0 , 
2397432.0 , 
2400154.0 , 
2403494.0 , 
2408186.0 , 
2414062.0 , 


,Index,GR_Utility,CP_Utility
1,0.0,2013442.0,0.0
2,1.0,225206.0,0.0
3,2.0,71632.0,0.0
4,3.0,44270.0,0.0
5,4.0,23630.0,0.0
6,5.0,9482.0,0.0
7,6.0,3420.0,0.0
8,7.0,2544.0,0.0
9,8.0,1704.0,0.0
10,9.0,2102.0,0.0


In [12]:
cps = [
    "NO_ARGS",
    # "all1", 
    #    "all3",
    #    "all5", 
    #    "all6", 
    #    "all7", "all8", "all10", "all12",
    # "ritnet","hotlab", 
    # "slam_lidar_steady", "slam_rgbd_steady"
    ]
# cps = ["vpi_sample_03_harris_corners","vpi_sample_11_fisheye","vpi_sample_12_optflow_lk_refined","klt_tracker","all5","ritnet","hotlab"]

for app in apps:
    print('        - ' + app + ':')
    # print('          - args:')
    for cp in cps:
        print('          - args: \''+ cp + '\'')
        print('            accel-sim-mem: 6G')


        - pbrtexture_2k:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - instancing_2k:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - render_passes_2k:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - sponza_2k:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - materials_2k:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - platformer_2k:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - demo_2k:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - pbrtexture_2k_lod0:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - instancing_2k_lod0:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - render_passes_2k_lod0:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - pbrtexture_4k:
          - args: 'NO_ARGS'
            accel-sim-mem: 6G
        - instancing_4k:
          - args: 'NO_ARGS'
            accel-sim-me